In [1]:
# Imports
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import numpy as np
from tqdm.notebook import tqdm

In [2]:
mapping = ["m", "p", "s"]


def convert_tile_to_number(tile):
    return int(tile[0] - 1) + 9 * mapping.index(tile[1])


def convert_number_to_tile(tile):
    return f"{tile % 9 + 1}{mapping[tile // 9]}"

In [7]:
# Data Frame Creation

with sqlite3.connect(os.path.abspath(os.path.abspath('') + "/../db/akarii.db")) as conn:
    df = pd.read_sql("SELECT * FROM riichi", conn)

df["winning_tiles"] = df["winning_tiles"].map(json.loads)

for t in range(27):
    df[f"{convert_number_to_tile(t)}_ron"] = df.winning_tiles.map(lambda x: t in x)

df = df.drop('winning_tiles', axis=1)

In [8]:
# DF Cleaning

df = df[df.tile < 27].reset_index(drop=True)  # Removing jihai riichi
df["tile_type"] = df["tile"] % 9  # Riichi tile type
df["tile_suit"] = df["tile"] // 9  # Riichi tile suit

In [10]:
# Calculations

def calculate_dealin_rates(DF=df):
    # [dropped tile, riichi tile, same suit / different suit]
    # dealin_rates = np.zeros((9, 9, 2))

    dealin_rates = np.zeros((9, 2))

    # for t_num in tqdm(range(9), leave=False):
    t_num = 4
    df_t_num = DF[DF.tile_type == t_num].reset_index(drop=True)

    for t_drop in range(9):
        # Same suit calculations
        same_suits = (df_t_num.tile_suit * 9 + t_drop).map(lambda x: f"{convert_number_to_tile(x)}_ron")
        res1 = df_t_num.to_numpy()[np.arange(len(df_t_num)), df_t_num.columns.get_indexer(same_suits)]
        # res = df_t_num.apply(lambda row: row[same_suits[row.name]], axis=1)

        different_1 = (((df_t_num.tile_suit + 1) % 3) * 9 + t_drop).map(
            lambda x: f"{convert_number_to_tile(x)}_ron")
        different_2 = (((df_t_num.tile_suit + 2) % 3) * 9 + t_drop).map(
            lambda x: f"{convert_number_to_tile(x)}_ron")
        res2 = df_t_num.to_numpy()[np.arange(len(df_t_num)), df_t_num.columns.get_indexer(different_1)]
        res3 = df_t_num.to_numpy()[np.arange(len(df_t_num)), df_t_num.columns.get_indexer(different_2)]

        res4 = np.concatenate([res2, res3])

        dealin_rates[t_drop][0] = res1.mean()
        dealin_rates[t_drop][1] = res4.mean()

    return dealin_rates


dealin_rates_all = calculate_dealin_rates()

In [12]:
print("Copy Pasteable - Same Suit")
print("\t".join(map(str, dealin_rates_all[:, 0] * 100)))
print("Copy Pasteable - Different Suit")
print("\t".join(map(str, dealin_rates_all[:, 1] * 100)))

Copy Pasteable - Same Suit
5.76295063072293	3.0319523596081193	3.6978933213805467	5.817378497790869	0.003201639239290517	6.022283409105462	4.02766216302747	3.2528654671191646	6.140744060959212
Copy Pasteable - Different Suit
4.786450662739323	6.119933405903823	7.427803035153999	9.188704616763783	9.369597233783697	9.424025100851637	7.53185631043094	6.384068643145291	4.948133444323494
